# Agenda

1. Recap + Q&A from yesterday
2. Exercise
3. Functions
    - What are they?
    - How do we define functions?
    - Arguments and parameters
    - Return values
    - Local vs. global variables
4. Modules + packages
    - What are modules/packages?
    - How can we use them?
    - Python's standard library
    - PyPI (the Python Package Index)
    - Installing/using packages with `pip`


# Recap

1. Dictionaries
    - Dicts are key-value stores
    - There are some rules for a dict
        - Every key must be immutable
        - Keys in a dict are unique
        - Every key has a value, and every value has a key
        - There are *no* restrictions on values; they can repeat, and can be any value we want or imagine
    - We retrieve via `[]`, indicating the key whose value we want
    - We can check if a key is in a dict with `in`
        - We cannot use `in` to find if a value is in a dict
    - We can assign to a dict using `[]`
        - If the key is new, then we add a key-value pair
        - If the key exists, then we update the value for that key
    - We can iterate over a dict
        - If we iterate over the dict object, we get the keys
        - The `dict.items` method returns `(key, value)` 2-element tuples, one at a time, when we iterate. This is my favorite way to iterate over a dict
    - Three paradigms for using dicts
        - Dicts are always mutable -- but these are three conventions for working with them, that I've seen a lot of
        - Define it, retrieve from it, but never update it -- use as a small in-memory database
        - Define a dict with keys and initial values; we never add/remove keys, but we do update the values to count things
        - Define an empty dict, adding keys (as needed) and values (as needed)

2. Files
    - To work with a file, we need to use `open`, which requests help and a file object from the OS
        - When we open a file, we can specify the "mode" as the second argument to `open`, after the filename
            - `'r'` (reading, the default)
            - `'w'` (writing, which removes any previous data / zeroes out the file we open if it exists)
            - `'a'` (append, like writing, but adds to the end of a file)
    - We can read from a file in at least three ways:
        - Invoke `read()`, getting the contents, but this is considered a bit dangerous
        - Invoke `read(n)`, which returns the next `n` characters, but this is annoying, because it doesn't stop at lines.